In [2]:
# Importación de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [5]:
# Análisis del DataFrame
# Carga el conjunto de datos
users_behavior_df = pd.read_csv('files/datasets/input/users_behavior.csv')
print(users_behavior_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


In [6]:
# Explora los primeros registros del conjunto de datos
display(users_behavior_df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [7]:
# Mejora del DataFrame
# Cambia el nombre de la columna mb_used
users_behavior_df.rename(columns={'mb_used': 'gb_used'}, inplace=True)

# Convertir todos los valores de la columna a GB y redondea a dos decimales
users_behavior_df['gb_used'] = (users_behavior_df['gb_used'] / 1024).round(2)

# Muestra los primeros registros del conjunto de datos después de los cambios
display(users_behavior_df.head())

,calls,minutes,messages,gb_used,is_ultra
0,40.0,311.90,83.0,19.45,0
1,85.0,516.75,56.0,22.16,0
2,77.0,467.66,86.0,20.57,0
3,106.0,745.53,81.0,8.24,1
4,66.0,418.74,1.0,14.16,0


In [8]:
#  Preparación de datos para el modelado
# Segmentación de datos en conjuntos de entrenamiento, validación y prueba
train_data, temp_data = train_test_split(users_behavior_df, test_size=0.3, random_state=54321)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=54321)

In [9]:
# Selección de características y variable objetivo
features = ['calls', 'minutes', 'messages', 'gb_used']
target = 'is_ultra'

In [10]:
# Entrenamiento de los modelos de clasificación con optimización de hiperparámetros
# Árbol de decisión
tree_params = {'max_depth': [3, 5, 7]}
tree_model = GridSearchCV(DecisionTreeClassifier(random_state=54321), tree_params, scoring='accuracy', cv=5)
tree_model.fit(train_data[features], train_data[target])

# Bosque aleatorio
forest_params = {'n_estimators': [50, 100, 200], 'max_depth': [5, 7, 9]}
forest_model = GridSearchCV(RandomForestClassifier(random_state=54321), forest_params, scoring='accuracy', cv=5)
forest_model.fit(train_data[features], train_data[target])

# Regresión logística
logistic_params = {'C': [0.1, 1, 10], 'penalty': ['l2']}
logistic_model = GridSearchCV(LogisticRegression(random_state=54321), logistic_params, scoring='accuracy', cv=5)
logistic_model.fit(train_data[features], train_data[target])

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=54321),
             param_grid={'C': [0.1, 1, 10], 'penalty': ['l2']},
             scoring='accuracy')

In [11]:
# Realización de predicciones en conjuntos de validación y prueba con los mejores modelos
# Realiza predicciones en el conjunto de validación y prueba utilizando el modelo de Árbol de Decisión
tree_predictions_valid = tree_model.predict(valid_data[features])
tree_predictions_test = tree_model.predict(test_data[features])

# Realiza predicciones en el conjunto de validación y prueba utilizando el modelo de Bosque Aleatorio
forest_predictions_valid = forest_model.predict(valid_data[features])
forest_predictions_test = forest_model.predict(test_data[features])

# Realiza predicciones en el conjunto de validación y prueba utilizando el modelo de Regresión Logística
logistic_predictions_valid = logistic_model.predict(valid_data[features])
logistic_predictions_test = logistic_model.predict(test_data[features])

In [12]:
# Evaluación  el rendimiento de los modelos
# Define una funcioón para la evaluación del rendimiento de los modelos
def evaluate_model(predictions_valid, predictions_test, actual_valid, actual_test, model_name):
    print(f"\nEvaluación del modelo {model_name}:")
    print("Exactitud en el conjunto de validación:", accuracy_score(actual_valid, predictions_valid))
    print("Exactitud en el conjunto de prueba:", accuracy_score(actual_test, predictions_test))
    print("Informe de clasificación en el conjunto de prueba:")
    print(classification_report(actual_test, predictions_test))
    
# Evaluación de los modelos
evaluate_model(tree_predictions_valid, tree_predictions_test, valid_data[target], test_data[target], "Árbol de Decisión")
evaluate_model(forest_predictions_valid, forest_predictions_test, valid_data[target], test_data[target], "Bosque Aleatorio")
evaluate_model(logistic_predictions_valid, logistic_predictions_test, valid_data[target], test_data[target], "Regresión Logística")


Evaluación del modelo Árbol de Decisión:
Exactitud en el conjunto de validación: 0.7738589211618258
Exactitud en el conjunto de prueba: 0.782608695652174
Informe de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.79      0.94      0.86       336
           1       0.75      0.43      0.55       147

    accuracy                           0.78       483
   macro avg       0.77      0.68      0.70       483
weighted avg       0.78      0.78      0.76       483


Evaluación del modelo Bosque Aleatorio:
Exactitud en el conjunto de validación: 0.7987551867219918
Exactitud en el conjunto de prueba: 0.7991718426501035
Informe de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.82      0.92      0.86       336
           1       0.74      0.52      0.61       147

    accuracy                           0.80       483
   macro avg       0.78      0.72      0.74

In [13]:
# Prueba de cordura
# Prueba de cordura 
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=54321)
dummy_clf.fit(train_data[features], train_data[target])
dummy_predictions = dummy_clf.predict(test_data[features])
accuracy_dummy = accuracy_score(test_data[target], dummy_predictions)

# Imprime resultados
print(f'\nAccuracy del Árbol de Decisión: {tree_model.best_score_}')
print(f'Parámetros del mejor Árbol de Decisión: {tree_model.best_params_}')
print()
print(f'Accuracy del Bosque Aleatorio: {forest_model.best_score_}')
print(f'Parámetros del mejor Bosque Aleatorio: {forest_model.best_params_}')
print()
print(f'Accuracy de la Regresión Logística: {logistic_model.best_score_}')
print(f'Parámetros de la mejor Regresión Logística: {logistic_model.best_params_}')
print()
print(f'Accuracy del DummyClassifier: {accuracy_dummy}')


Accuracy del Árbol de Decisión: 0.7945686711210096
Parámetros del mejor Árbol de Decisión: {'max_depth': 7}

Accuracy del Bosque Aleatorio: 0.8145805493689681
Parámetros del mejor Bosque Aleatorio: {'max_depth': 9, 'n_estimators': 50}

Accuracy de la Regresión Logística: 0.752781984657263
Parámetros de la mejor Regresión Logística: {'C': 0.1, 'penalty': 'l2'}

Accuracy del DummyClassifier: 0.6956521739130435
